<a href="https://colab.research.google.com/github/jasonantonio10/IndoSROIE---related/blob/main/Finding_Skew_angle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from scipy.ndimage import interpolation as inter
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:

def correct_skew(image, delta=1, limit=90):
  ''' delta = change in angle
      limit = maximum angle
      # Projection Profile Implementation from https://stackoverflow.com/questions/59660933/how-to-de-skew-a-text-image-and-retrieve-the-new-bounding-box-of-that-image-pyth 
  '''
    def determine_score(arr, angle):
        data = inter.rotate(arr, angle, reshape=False, order=0)
        histogram = np.sum(data, axis=1, dtype=float)
        '''
        print(f'data: {data}')
        print(f'data shape: {data.shape}')
        print(f'Histogram: {histogram}')
        print(f'Histogram[1:]: {histogram[1:]}')
        print(f'Histogram[:-1]: {histogram[:-1]}')
        '''
        score = np.sum((histogram[1:] - histogram[:-1]) ** 2, dtype=float)
        return histogram, score

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1] 

    scores = []
    angles = np.arange(-limit, limit + delta, delta)
    for angle in angles:
        histogram, score = determine_score(thresh, angle)
        scores.append(score)

    best_angle = angles[scores.index(max(scores))]

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, best_angle, 1.0)
    corrected = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, \
            borderMode=cv2.BORDER_REPLICATE)

    return best_angle, corrected


In [ ]:
image = cv2.imread('/content/new doc 2022-06-22 22.31.13n-1.jpg')
angle, corrected = correct_skew(image)
print('Skew angle:', angle)
cv2_imshow(corrected)